In [ ]:
import xgboost as xgb
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
# 1. Chargement du Parquet
df = pd.read_parquet("dataset_full.parquet")

# 2. Pré-processing identique (Types et ESS)
df['Catégorie juridique de l\'unité légale'] = df['Catégorie juridique de l\'unité légale'].astype(str)
df['age_estime'] = df['age_estime'].astype(float)
df['Tranche_effectif_num'] = df['Tranche_effectif_num'].fillna(0).astype(float)
df['is_ess'] = df['Economie sociale et solidaire unité légale'].map({'O': 1, 'N': 0}).fillna(0).astype(int)

# 3. Géographie (Réutilisation de la logique de risque)
df['Code du département de l\'établissement'] = df['Code du département de l\'établissement'].astype(str).str.zfill(2)
# Note : Pour être parfait, il faudrait charger la 'dep_risk_map' sauvegardée du train
dep_risk_map = df.groupby("Code du département de l'établissement")["fermeture"].mean()
df['risque_departemental'] = df['Code du département de l\'établissement'].map(dep_risk_map)

# 4. One-Hot Encoding initial
df_final = pd.get_dummies(
    df, 
    columns=['libelle_section_ape', 'Catégorie juridique de l\'unité légale'], 
    prefix=['APE', 'CJ'],
    drop_first=True,
    dtype=int 
)

# --- 5. ALIGNEMENT DYNAMIQUE  ---

model = xgb.Booster()
model.load_model("xgboost_v2.json")
expected_features = model.feature_names

# On identifie ce qui est "Rare" pour CE nouveau dataset par rapport au modèle
current_cols = df_final.columns
rare_ape_to_merge = [c for c in current_cols if c.startswith('APE_') and c not in expected_features]
rare_cj_to_merge = [c for c in current_cols if c.startswith('CJ_') and c not in expected_features]

# Fusion dans "Autres" si nécessaire
if 'APE_Autres_Secteurs' in expected_features:
    df_final['APE_Autres_Secteurs'] = df_final[rare_ape_to_merge].any(axis=1).astype(int) if rare_ape_to_merge else 0

if 'CJ_Autres_Status' in expected_features:
    df_final['CJ_Autres_Status'] = df_final[rare_cj_to_merge].any(axis=1).astype(int) if rare_cj_to_merge else 0

# --- L'ÉTAPE CLÉ : REINDEX ---

X_inference = df_final.reindex(columns=expected_features, fill_value=0)

print(f"📊 Colonnes attendues : {len(expected_features)}")
print(f"📊 Colonnes finales : {X_inference.shape[1]}")

In [ ]:
X_inference.head()

In [ ]:
# 1. Création de la DMatrix (obligatoire pour XGBoost)
dmatrix_inference = xgb.DMatrix(X_inference)

# 2. Prédiction du Mu (le score brut en log-time)
preds_mu = model.predict(dmatrix_inference)

# --- 2. FONCTION DE CALCUL DU RISQUE (MISE À JOUR AVEC CALIBRATION) ---
def calculer_risque_calibre(mu, annees, sigma=1.5):
    calibration_factor = 2.5 
    # On applique le décalage de 2.5 comme dans ton train
    z = (np.log(annees) - (mu - calibration_factor)) / sigma
    
    # Formule Sigmoïde inversée pour le risque (1 / (1 + exp(-z)))
    return (1 / (1 + np.exp(-z))) * 100

# --- 3. GÉNÉRATION DES SCORES (AVEC LES PARAMÈTRES DU TRAIN) ---

df['Risque_1an'] = calculer_risque_calibre(preds_mu, 1, sigma=1.5)
df['Risque_2ans'] = calculer_risque_calibre(preds_mu, 2, sigma=1.5)
df['Risque_3ans'] = calculer_risque_calibre(preds_mu, 3, sigma=1.5)

# --- 4. DIAGNOSTIC EXPERT (Inchangé) ---
def determiner_statut(row):
    if row['Risque_1an'] > 15:
        return "🔴 CRITIQUE (Immédiat)"
    elif row['Risque_2ans'] > 25:
        return "🟠 ALERTE (Moyen terme)"
    elif row['Risque_3ans'] > 35:
        return "🟡 FRAGILE (Long terme)"
    else:
        return "🟢 SOLIDE"

df['Diagnostic_Final'] = df.apply(determiner_statut, axis=1)

# --- 5. AFFICHAGE DES RÉSULTATS ---
colonnes_affichage = [
    'SIREN', 
    "Dénomination de l'unité légale", 
    'Risque_1an', 
    'Risque_2ans', 
    'Risque_3ans', 
    'Diagnostic_Final'
]

print("📊 Analyse de risque CALIBRÉE sur 3 ans terminée")
display(df[colonnes_affichage].round(2).tail(100))

In [ ]:
# 1. Utilise le nom de l'objet que tu as chargé au début (probablement 'model')
preds_log_time = model.predict(dmatrix_inference)

# 2. Ta fonction de risque (sans le décalage de 2.5 qui faussait tout)
def calculer_risque_propre(mu, annees, sigma=1.5):
    # Formule AFT Logistique standard : Z = (log(t) - mu) / sigma
    z = (np.log(annees) - mu) / sigma
    # Risque = 1 / (1 + exp(-z))
    return (1 / (1 + np.exp(-z))) * 100

# 3. Application des calculs
df['Risque_1an'] = calculer_risque_propre(preds_log_time, 1)
df['Risque_2ans'] = calculer_risque_propre(preds_log_time, 2)
df['Risque_3ans'] = calculer_risque_propre(preds_log_time, 3)

# 4. Affichage pour vérification
display(df[['SIREN', 'Dénomination de l\'unité légale', 'Risque_1an', 'Risque_3ans']].head(20))

In [ ]:


# Configuration du style
sns.set_theme(style="whitegrid")
plt.figure(figsize=(15, 6))

# --- GRAPHIQUE 1 : Distribution du Risque à 1 an ---
plt.subplot(1, 2, 1)
sns.histplot(df['Risque_1an'], bins=30, kde=True, color='#2ecc71')
plt.axvline(df['Risque_1an'].mean(), color='red', linestyle='--', label=f"Moyenne: {df['Risque_1an'].mean():.1f}%")
plt.title('Répartition du Risque à 1 an')
plt.xlabel('% de Risque de fermeture')
plt.ylabel('Nombre d\'entreprises')
plt.legend()

# --- GRAPHIQUE 2 : Comparaison Risque 1 an vs 3 ans ---
plt.subplot(1, 2, 2)
sns.scatterplot(data=df, x='Risque_1an', y='Risque_3ans', alpha=0.5, hue='Risque_1an', palette='viridis')
plt.plot([0, 60], [0, 60], color='red', linestyle='--') # Ligne d'égalité
plt.title('Évolution du Risque (1 an vs 3 ans)')
plt.xlabel('Risque à 1 an (%)')
plt.ylabel('Risque à 3 ans (%)')

plt.tight_layout()
plt.show()

# --- RÉSUMÉ STATISTIQUE ---
print("📊 RÉSUMÉ DES PRÉDICTIONS :")
print(df[['Risque_1an', 'Risque_2ans', 'Risque_3ans']].describe().round(2))

---

##### Features importances

In [ ]:
xgb.plot_importance(model, max_num_features=10)
plt.show()

#### Visu des gains

In [ ]:
plt.figure(figsize=(10, 8))
xgb.plot_importance(model, max_num_features=15, importance_type='gain')
plt.show()

##### Test de réactivité

In [ ]:
# On prend une entreprise au hasard
test_row = X_inference.iloc[[0]].copy()

print(f"--- TEST DE RÉACTIVITÉ DU MODÈLE (SIREN: {df.iloc[0]['SIREN']}) ---")

for age_test in [1.0, 5.0, 20.0]:
    test_row['age_au_diagnostic'] = age_test
    
    # Prédiction Mu
    mu_test = model.predict(xgb.DMatrix(test_row))[0]
    
    # Risque à 1 an (avec notre fonction propre)
    risk = calculer_risque_propre(mu_test, 1, sigma=1.5)
    
    print(f"Pour un âge de {age_test:2.0f} ans -> Risque à 1 an : {risk:.2f}%")

In [ ]:
# Test avec un Sigma plus serré pour forcer la nuance
def calculer_risque_nuance(mu, annees, sigma=0.7): # Sigma divisé par 2
    z = (np.log(annees) - mu) / sigma
    return (1 / (1 + np.exp(-z))) * 100

# Test sur ton entreprise de 5 ans qui donnait 0%
mu_exemple = 5.2  # (Valeur hypothétique proche de tes résultats)
print(f"Risque à 5 ans (Sigma 1.5) : {calculer_risque_propre(mu_exemple, 1, 1.5):.2f}%")
print(f"Risque à 5 ans (Sigma 0.7) : {calculer_risque_nuance(mu_exemple, 1, 0.7):.2f}%")